In [1]:
import collections
import os
import string
import sys

import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
from pprint import pprint
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lidia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
data_path = "../data/txt/"

# Choix d'une décennie et du nombre de clusters

In [4]:
N_CLUSTERS = 5

# Chargement des fichiers de la décennie

In [5]:
files = [f for f in sorted(os.listdir(data_path)) if f"_{all}" in f]

In [6]:
# Exemple de fichiers
files[:2028]

[]

In [7]:
texts = [open(data_path + f, encoding = 'utf-8').read() for f in files]

In [8]:
# Exemple de textes
texts[0][:400]

IndexError: list index out of range

# Vectorisation du texte

In [10]:
def process_text(text, stem=True):
    """ Tokenize text and remove punctuation """
    text = text.translate(string.punctuation)
    tokens = word_tokenize(text)
    return tokens

In [11]:
vectorizer = TfidfVectorizer(tokenizer=process_text,
                                stop_words=stopwords.words('french'),
                                max_df=0.5,
                                min_df=0.1,
                                lowercase=True)

In [12]:
%time tfidf_vectors = vectorizer.fit_transform(texts)

Wall time: 1min 8s


In [13]:
tfidf_vectors

<97x8985 sparse matrix of type '<class 'numpy.float64'>'
	with 184711 stored elements in Compressed Sparse Row format>

In [14]:
# Exemple de vecteur TFIDF
pd.Series(
    tfidf_vectors[0].toarray()[0],
    index=vectorizer.get_feature_names()
    ).sort_values(ascending=False)

file          0.401581
galerie       0.267132
partoes       0.212588
bateau        0.152975
bancs         0.143414
                ...   
mensuelle     0.000000
mentale       0.000000
mentionne     0.000000
mentionner    0.000000
#             0.000000
Length: 8985, dtype: float64

# Comprendre les vecteurs et leurs "distances"

In [15]:
from scipy.spatial.distance import cosine

In [16]:
cosine([1, 2, 3], [1, 2, 3])

0.0

In [17]:
cosine([1, 2, 3], [1, 2, 2])

0.02004211298777725

In [18]:
cosine([1, 2, 3], [2, 2, 2])

0.07417990022744858

In [19]:
tfidf_array = tfidf_vectors.toarray()

In [20]:
tfidf_array[0]

array([0.        , 0.        , 0.        , ..., 0.        , 0.        ,
       0.00909305])

In [21]:
texts[0][:200]

"LLETIN COMMUNAL.\nANNÉE 1850.\n\nPREMIER\n\nSEMESTRE,\n\nBRUXELLES,\nOIS-WITTOI'CK\n\n,\n\nIMPlUMtVR\n\nDE\n\nLA\n\nVILLE,\n\n1U K\n\n\\V\n\nl\n\n\x0clïi\n\n\x0cVILLE\n\nDE\n\nBRUXELLES.\n\nBULLETIN COMMUNAL.\nANNÉE 1 8 5 0 . —\n\nN° 1.\n\nTravau"

In [22]:
tfidf_array[0]

array([0.        , 0.        , 0.        , ..., 0.        , 0.        ,
       0.00909305])

In [23]:
cosine(tfidf_array[0], tfidf_array[1])

0.7699143026350066

# Clustering des vecteurs TFIDF

Article intéressant sur le KMeans clustering :
- https://medium.com/dataseries/k-means-clustering-explained-visually-in-5-minutes-b900cc69d175

In [24]:
km_model = KMeans(n_clusters=N_CLUSTERS)

In [25]:
km_model.fit(tfidf_vectors)

KMeans(n_clusters=5)

In [26]:
clustering = collections.defaultdict(list)

for idx, label in enumerate(km_model.labels_):
    clustering[label].append(files[idx])

In [27]:
pprint(dict(clustering))

{0: ['Bxl_1854_Tome_II1_Part_1.txt',
     'Bxl_1855_Tome_I1_Part_3.txt',
     'Bxl_1855_Tome_I1_Part_4.txt',
     'Bxl_1855_Tome_I1_Part_5.txt',
     'Bxl_1855_Tome_I1_Part_6.txt',
     'Bxl_1855_Tome_II1_Part_1.txt',
     'Bxl_1855_Tome_II1_Part_3.txt',
     'Bxl_1856_Tome_I1_Part_1.txt',
     'Bxl_1856_Tome_I1_Part_2.txt',
     'Bxl_1856_Tome_I1_Part_3.txt',
     'Bxl_1856_Tome_I1_Part_4.txt',
     'Bxl_1856_Tome_I1_Part_5.txt',
     'Bxl_1856_Tome_II1_Part_1.txt',
     'Bxl_1856_Tome_II1_Part_6.txt',
     'Bxl_1857_Tome_I1_Part_1.txt',
     'Bxl_1857_Tome_I1_Part_2.txt',
     'Bxl_1857_Tome_I1_Part_3.txt',
     'Bxl_1857_Tome_I1_Part_4.txt',
     'Bxl_1857_Tome_II1_Part_1.txt'],
 1: ['Bxl_1857_Tome_II1_Part_5.txt',
     'Bxl_1857_Tome_II1_Part_6.txt',
     'Bxl_1858_Tome_I1_Part_1.txt',
     'Bxl_1858_Tome_I1_Part_2.txt',
     'Bxl_1858_Tome_I1_Part_3.txt',
     'Bxl_1858_Tome_I1_Part_4.txt',
     'Bxl_1858_Tome_II1_Part_1.txt',
     'Bxl_1858_Tome_II1_Part_2.txt',
     'Bxl_1858_To